In [318]:
from sdim import *
import cirq # using cirq to visualize the circuit
import numpy as np

### Encoding
We encode one qubit of information distributed amongst three using CNOT's. This is only protected against a single X error.

In [319]:
# Define the circuit
circuit = Circuit(5, 2) # Three qubit repitition code with two ancilla qubits
circuit.add_gate('CNOT', 0, 1)
circuit.add_gate('CNOT', 0, 2)

In [320]:
def insert_x_error(circuit, p=0.5):
    if np.random.rand() < p:
        i = np.random.choice(3)  # choose a random qubit from 0, 1, 2
        circuit.add_gate('X', i)
        print(f'Error inserted on qubit {i}')
    else:
        print('No error inserted')
    return
insert_x_error(circuit, p=0.9)

Error inserted on qubit 0


### Syndrome Measurements
We check the operators $Z\otimes Z \otimes I$ and $I\otimes Z \otimes Z$, but these are the generators and so other syndromes can be measured, leading to different circuit constructions.

In [321]:
# ZZI
circuit.add_gate('CNOT', 0, 3)
circuit.add_gate('CNOT', 1, 3)
# IZZ
circuit.add_gate('CNOT', 1, 4)
circuit.add_gate('CNOT', 2, 4)
# Measure
circuit.add_gate('MEASURE', 3)
circuit.add_gate('MEASURE', 4)

program = Program(circuit)
result = program.simulate()
print(result)

[Measured qudit (3) as (1) and was deterministic, Measured qudit (4) as (0) and was deterministic]


In [322]:
cirq_circuit = circuit_to_cirq_circuit(circuit)
print(cirq_circuit)

Measurement on qudit 3
Measurement on qudit 4
                                                                                      ┌─────────────────┐
0 (d=2): ───CNOT_2_control───CNOT_2_control───X_2───CNOT_2_control─────────────────────────────────────────────────────────────────
            │                │                      │
1 (d=2): ───CNOT_2_target────┼──────────────────────┼────────────────CNOT_2_control────CNOT_2_control──────────────────────────────
                             │                      │                │                 │
2 (d=2): ────────────────────CNOT_2_target──────────┼────────────────┼─────────────────┼────────────────────CNOT_2_control─────────
                                                    │                │                 │                    │
3 (d=2): ───────────────────────────────────────────CNOT_2_target────CNOT_2_target─────┼─────────────I_2────┼──────────────────────
                                                                   

The following table is the syndrome and the corresponding error
\begin{array}{|c|c|}
\hline
\text{Syndrome $(q_3, q_4)$} & \text{Error} \\
\hline
00 & \text{None} \\
01 & X \text{ on qubit 2} \\
10 & X \text{ on qubit 0} \\
11 & X \text{ on qubit 1} \\
\hline
\end{array}

### Moving to qutrits
The concept is the same, measuring syndromes to calculate the parity. The new syndrome mesaurements are $Z\otimes Z^{-1} \otimes I$ and $I \otimes Z \otimes Z^{-1}$. These syndromes also have to detect more types of 'bit' flips, namely $X$ and $X^2$, where the ternary Pauli operator is no longer Hermitian.

In [323]:
qutrit_circuit = Circuit(5, 3)
qutrit_circuit.add_gate('CNOT', 0, 1) # Called CNOT but turns into SUM gate
qutrit_circuit.add_gate('CNOT', 0, 2) 

In [324]:
def insert_qutrit_x_error(circuit, p=0.5):
    if np.random.rand() < p:
        i = np.random.choice(3)  # choose a random qubit from 0, 1, 2
        error_order = np.random.choice([1, 2])  # choose between 1 and 2
        for _ in range(error_order):
            circuit.add_gate('X', i)
        print(f'X^{error_order} error inserted on qubit {i}')
    else:
        print('No error inserted')
    return
insert_qutrit_x_error(qutrit_circuit, p=0.8)

X^2 error inserted on qubit 1


In [325]:
# ZZ^2I
qutrit_circuit.add_gate('CNOT', 0, 3)
qutrit_circuit.add_gate('CNOT', 1, 3)
qutrit_circuit.add_gate('CNOT', 1, 3)
# IZZ^2
qutrit_circuit.add_gate('CNOT', 1, 4)
qutrit_circuit.add_gate('CNOT', 2, 4)
qutrit_circuit.add_gate('CNOT', 2, 4)
# Measure
qutrit_circuit.add_gate('MEASURE', 3)
qutrit_circuit.add_gate('MEASURE', 4)

qutrit_program = Program(qutrit_circuit)
qutrit_result = qutrit_program.simulate()
print(qutrit_result)

[Measured qudit (3) as (1) and was deterministic, Measured qudit (4) as (2) and was deterministic]


In [326]:
cirq_qutrit_circuit = circuit_to_cirq_circuit(qutrit_circuit)
print(cirq_qutrit_circuit)

Measurement on qudit 3
Measurement on qudit 4
                             ┌─────────────────┐   ┌─────────────────┐                                     ┌─────────────────┐
0 (d=3): ───CNOT_3_control────CNOT_3_control───────────CNOT_3_control────────────────────────────────────────────────────────────────────────────────────────────────────
            │                 │                        │
1 (d=3): ───CNOT_3_target─────┼─────────────X_3─────X_3┼─────────────────CNOT_3_control───CNOT_3_control────CNOT_3_control───────────────────────────────────────────────
                              │                        │                 │                │                 │
2 (d=3): ─────────────────────CNOT_3_target────────────┼─────────────────┼────────────────┼─────────────────┼────────────────────CNOT_3_control───CNOT_3_control─────────
                                                       │                 │                │                 │                    │                │
3 

\begin{array}{|c|c|}
\hline
\text{Syndrome $(q_3, q_4)$} & \text{Error} \\
\hline
00 & \text{None} \\
10 & X \text{ on qubit 0} \\
20 & X^2 \text{ on qubit 0} \\
21 & X \text{ on qubit 1} \\
12 & X^2 \text{ on qubit 1} \\
02 & X \text{ on qubit 2} \\ 
01 & X^2 \text{ on qubit 2} \\ 
\hline
\end{array}

### Moving to ququarts
Things get a little strange in composite dimensions. The syndrome mesaurements are still $Z\otimes Z^{-1} \otimes I$ and $I \otimes Z \otimes Z^{-1}$.

In [327]:
ququart_circuit = Circuit(5, 3)
ququart_circuit.add_gate('CNOT', 0, 1) # Called CNOT but turns into SUM gate
ququart_circuit.add_gate('CNOT', 0, 2) 

In [328]:
def insert_ququart_x_error(circuit, p=0.5):
    if np.random.rand() < p:
        i = np.random.choice(3)  # choose a random qubit from 0, 1, 2
        error_order = np.random.choice([1, 2, 3])  # choose between 1 and 2
        for _ in range(error_order):
            circuit.add_gate('X', i)
        print(f'X^{error_order} error inserted on qubit {i}')
    else:
        print('No error inserted')
    return
insert_ququart_x_error(ququart_circuit, p=0.8)

X^1 error inserted on qubit 1


In [329]:
# ZZ^3I
ququart_circuit.add_gate('CNOT', 0, 3)
ququart_circuit.add_gate('CNOT', 1, 3)
ququart_circuit.add_gate('CNOT', 1, 3)
ququart_circuit.add_gate('CNOT', 1, 3)
# IZZ^3
ququart_circuit.add_gate('CNOT', 1, 4)
ququart_circuit.add_gate('CNOT', 2, 4)
ququart_circuit.add_gate('CNOT', 2, 4)
ququart_circuit.add_gate('CNOT', 2, 4)
# Measure
ququart_circuit.add_gate('MEASURE', 3)
ququart_circuit.add_gate('MEASURE', 4)

ququart_program = Program(ququart_circuit)
ququart_result = ququart_program.simulate()
print(ququart_result)

[Measured qudit (3) as (0) and was deterministic, Measured qudit (4) as (1) and was deterministic]


In [330]:
cirq_ququart_circuit = circuit_to_cirq_circuit(ququart_circuit)
print(cirq_ququart_circuit)

Measurement on qudit 3
Measurement on qudit 4
                             ┌─────────────────┐                                                                       ┌─────────────────┐
0 (d=3): ───CNOT_3_control────CNOT_3_control───────CNOT_3_control─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
            │                 │                    │
1 (d=3): ───CNOT_3_target─────┼─────────────X_3────┼────────────────CNOT_3_control───CNOT_3_control───CNOT_3_control────CNOT_3_control────────────────────────────────────────────────────────────────
                              │                    │                │                │                │                 │
2 (d=3): ─────────────────────CNOT_3_target────────┼────────────────┼────────────────┼────────────────┼─────────────────┼────────────────────CNOT_3_control───CNOT_3_control───CNOT_3_control─────────
                                           